In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


In [13]:
data = pd.read_csv("..\datasets\ObesityDataSet_raw_and_data_sinthetic.csv")
print(data)

      Gender        Age    Height      Weight family_history_with_overweight  \
0     Female  21.000000  1.620000   64.000000                            yes   
1     Female  21.000000  1.520000   56.000000                            yes   
2       Male  23.000000  1.800000   77.000000                            yes   
3       Male  27.000000  1.800000   87.000000                             no   
4       Male  22.000000  1.780000   89.800000                             no   
...      ...        ...       ...         ...                            ...   
2106  Female  20.976842  1.710730  131.408528                            yes   
2107  Female  21.982942  1.748584  133.742943                            yes   
2108  Female  22.524036  1.752206  133.689352                            yes   
2109  Female  24.361936  1.739450  133.346641                            yes   
2110  Female  23.664709  1.738836  133.472641                            yes   

     FAVC  FCVC  NCP       CAEC SMOKE  

In [14]:
data.isnull().any()

Gender                            False
Age                               False
Height                            False
Weight                            False
family_history_with_overweight    False
FAVC                              False
FCVC                              False
NCP                               False
CAEC                              False
SMOKE                             False
CH2O                              False
SCC                               False
FAF                               False
TUE                               False
CALC                              False
MTRANS                            False
NObeyesdad                        False
dtype: bool

In [15]:
X = data.drop("NObeyesdad", axis=1)
print(X)

      Gender        Age    Height      Weight family_history_with_overweight  \
0     Female  21.000000  1.620000   64.000000                            yes   
1     Female  21.000000  1.520000   56.000000                            yes   
2       Male  23.000000  1.800000   77.000000                            yes   
3       Male  27.000000  1.800000   87.000000                             no   
4       Male  22.000000  1.780000   89.800000                             no   
...      ...        ...       ...         ...                            ...   
2106  Female  20.976842  1.710730  131.408528                            yes   
2107  Female  21.982942  1.748584  133.742943                            yes   
2108  Female  22.524036  1.752206  133.689352                            yes   
2109  Female  24.361936  1.739450  133.346641                            yes   
2110  Female  23.664709  1.738836  133.472641                            yes   

     FAVC  FCVC  NCP       CAEC SMOKE  

In [18]:
Y = data["NObeyesdad"]
print(pd.get_dummies(Y, columns="NObeyesdad"))

      Insufficient_Weight  Normal_Weight  Obesity_Type_I  Obesity_Type_II  \
0                   False           True           False            False   
1                   False           True           False            False   
2                   False           True           False            False   
3                   False          False           False            False   
4                   False          False           False            False   
...                   ...            ...             ...              ...   
2106                False          False           False            False   
2107                False          False           False            False   
2108                False          False           False            False   
2109                False          False           False            False   
2110                False          False           False            False   

      Obesity_Type_III  Overweight_Level_I  Overweight_Level_II  
0        

In [17]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.25,
    random_state=42,
)

clf.fit(X_train, Y_train)
test_df = pd.DataFrame()
test_df["actual"] = Y_test
test_df["predicted"] = clf.predict(X_test)

ValueError: could not convert string to float: 'Female'